In [2]:
# Importing necessary libraries
import pandas as pd               
import numpy as np                
import matplotlib.pyplot as plt   
import os                         
import joblib                     
import hopsworks                  

from OPS.Feature_pipeline import amd_fg   #Loading in the amd_fg 
#Making the notebook able to fetch from the .env file
from dotenv import load_dotenv
import os

load_dotenv()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
Connected. Call `.close()` to terminate connection gracefully.
         date  1. open   2. high    3. low  4. close   5. volume ticker
0  2024-10-04  166.533  171.2100  164.4200    170.90  44640296.0    AMD
1  2024-10-03  159.710  166.7894  159.4000    162.85  29598787.0    AMD
2  2024-10-02  159.310  163.1000  158.6689    159.78  24630431.0    AMD
3  2024-10-01  164.480  165.4000  158.0800    159.75  31831860.0    AMD
4  2024-09-30  163.100  165.0800  162.1800    164.08  21920300.0    AMD
2024-10-13 10:48:51,930 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.

2024-10-13 10:48:51,930 WARNING: FeatureGroupWarning: The ingested dataframe contains feature names with spaces: `['1. open', '2. high', '3. low', '4. close', '5. volume']`. Feature names are sanitized to use underscore '_' in the 

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/564374/featurestores/560197/featuregroups). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270089,"usrMsg":"project: MyMLCourse, featurestoreId: 560197","errorMsg":"The feature group you are trying to create does already exist."}', error code: 270089, error msg: The feature group you are trying to create does already exist., user msg: project: MyMLCourse, featurestoreId: 560197

In [3]:
#Getting connected to hopsworks 
api_key = os.environ.get('hopsworks_api') 
project = hopsworks.login(api_key_value=api_key) 
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
#Defining the function to create feature view

def create_stocks_feature_view(fs, version):

    # Loading in the feature groups
    amd_fg = fs.get_feature_group('amd_stock', version=7)
    

    # Defining the query
    ds_query = amd_fg.select(['date', 'open', 'close'])\
        

    # Creating the feature view
    feature_view = fs.create_feature_view(
        name='amd_stocks_fv',
        query=ds_query,
        labels=['open']
    )

    return feature_view, amd_fg

In [5]:
import re

# Function to clean column names
def clean_column_name(col):
    # Convert to lowercase
    col = col.lower()
    # Replace any invalid characters with an underscore
    col = re.sub(r'[^a-z0-9_]', '_', col)
    # If the column name doesn't start with a letter, prefix it with 'f_'
    if not re.match(r'^[a-z]', col):
        col = 'f_' + col
    return col

# Apply the function to all column names
amd_fg.columns = [clean_column_name(col) for col in amd_fg.columns]

print("Cleaned column names:")
print(amd_fg.columns.tolist())

NameError: name 'amd_fg' is not defined

In [6]:
#Creating the feature view
try:
    feature_view = fs.get_feature_view("amd_stocks_fv", version=7)
    amd_fg = fs.get_feature_group('amd_stock', version=7)
except:
    feature_view, amd_fg_fg = create_stocks_feature_view(fs, 7)

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/564374/featurestores/560197/featuregroups/amd_stock). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":270009,"usrMsg":"feature group name: amd_stock feature group version: 1","errorMsg":"Featuregroup wasn\'t found."}', error code: 270009, error msg: Featuregroup wasn't found., user msg: feature group name: amd_stock feature group version: 1

In [1]:
#Defining a function to get fixed data from the feature view
def fix_data_from_feature_view(df,start_date,end_date):
    df = df.sort_values("date")
    df = df.reset_index()
    df = df.drop(columns=["index"])

    # Create a boolean mask for rows that fall within the date range
    mask = (pd.to_datetime(df['date']) >= pd.to_datetime(start_date)) & (pd.to_datetime(df['date']) <= pd.to_datetime(end_date))
    len_df = np.shape(df)
    df = df[mask] # Use the boolean mask to filter the DataFrame
    print('From shape {} to {} after cropping to given date range: {} to {}'.format(len_df,np.shape(df),start_date,end_date))

    # Get rid off all non-business days
    isBusinessDay, is_open = extract_business_day(start_date,end_date)
    is_open = [not i for i in is_open] # Invert the mask to be able to drop all non-buisiness days

    filtered_df = df.drop(df[is_open].index) # Use the mask to filter the rows of the DataFrame
    print('From shape {} to {} after removing non-business days'.format(np.shape(df),np.shape(filtered_df)))
    print(filtered_df)
    